In [1]:
import numpy as np


In [87]:
class Value:
    def __init__(self, value, op = None, prev1 = None, prev2 = None): #operation of how i got here
        self.value = value
        self.grad = 0
        self.op = op
        self.prev1 = prev1
        self.prev2 = prev2
    
    def __repr__(self): 
        if self.prev2:
            if isinstance(self.prev2, Value):
                return f"|Value : {self.value}, grad = {self.grad}|, prev1 : {self.prev1.value}, prev2 : {self.prev2.value}"
            else:
                return f"|Value : {self.value}, grad = {self.grad}|, prev1 : {self.prev1.value}, prev2 : {self.prev2}"
        else:
            return f"|Value : {self.value}, grad = {self.grad}|, prev1 : {self.prev1}, prev2 : {self.prev2}"

    # def draw_recurse(self, string):
    #     if self.prev1 is None and self.prev2 is None:
    #         string += self.value
    #     elif self.prev2 is None:
    #         if not isinstance(self.prev1, Value):
    #             string += self.prev1
            
    

    def __neg__(self):
        return self * (-1)
    
    def __add__(self, other):
        return Value(self.value + other.value, op = '+' , prev1 = self, prev2 = other) if isinstance(other, Value) else Value(self.value + other, op = '+', prev1 = self)

    def backward_add(self):
        if isinstance(self.prev1, Value):
            self.prev1.grad = self.grad
            if (isinstance(self.prev2, Value)):
                self.prev2.grad = self.grad
                self.prev2.backward()
            self.prev1.backward()
    
    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        return Value(self.value * other.value, op = '*', prev1 = self, prev2 = other) if isinstance(other, Value) else Value(self.value * other, op = '*', prev1 = self, prev2 = other)

    def backward_mul(self):
        if isinstance(self.prev1, Value):
            if isinstance(self.prev2, Value):
                self.prev1.grad = self.prev2.value
                self.prev2.grad = self.prev1.value
                self.prev2.backward()
            else:
                self.prev1.grad = self.prev2
            self.prev1.backward()
            

    def __rmul__(self, other):
        return self * other

    def backward(self): # for calculating the grad 
        if self.op == '+':
            self.backward_add()
        elif self.op == '*':
            self.backward_mul()
                        

In [85]:
a = np.random.randn(1,3)

for n in a[0] :
    print(n.item())

0.19116775658765706
-0.3191220171594739
-0.8365869052278189


In [95]:
def RelU(x):
    return x if x>0 else 0

class Neuron:
    def __init__(self, numInputs):
        self.weights = np.random.randn(numInputs).reshape(1, numInputs)
        for weight in self.weights[0]:
            weight = Value(weight.item())
        self.bias = Value(np.random.randn(1).item())

    def getOutput(self, inputs, len_inputs):
        result = 0
        if len_inputs == len(self.weights):
            for i in range(len_inputs):
                result += self.weights[i].value * inputs[i]
            result += self.bias.value
            return RelU(result)
        else:
            return None

class Layer:
    def __init__(self, numNeuronsPrevLayer, numNeurons):
        self.neurons = []
        for i in range(numNeurons):
            self.neurons.append(Neuron(numNeuronsPrevLayer))

    def getOuts(self, inputs):
        ln = len(inputs)
        Outs = []
        if ln == len(self.neurons):
            for neuron in self.neurons:
                Outs.append(neuron.getOutput(inputs, ln))
            return Outs
        else:
            return None

class MLP:
    def __init__(self, numNeuronsEachLayer, numInputs):
        self.layers=[Layer(numInputs, numNeuronsEachLayer[0])]
        for i in range(1, len(numNeuronsEachLayer)):
            self.layers.append(Layer(numNeuronsEachLayer[i-1], numNeuronsEachLayer[i]))

    def getPrediction(self, inputs, target):
        for layer in self.layers:
            inputs = layer.getOuts(inputs)
        return inputs
            
        

In [98]:
mlp = MLP([2,2,1], 2)
print(mlp.getPrediction([0.5,1.5], -1))

None


In [62]:
a = Value(3)
b = 5 * a
c = Value(-2)
d = b + c
d.grad = 1
d.backward()

In [63]:
print(a)
print(b)
print(c)
print(d)

|Value : 3, grad = 5|, prev1 : None, prev2 : None
|Value : 15, grad = 1|, prev1 : 3, prev2 : 5
|Value : -2, grad = 1|, prev1 : None, prev2 : None
|Value : 13, grad = 1|, prev1 : 15, prev2 : -2


In [51]:
a = None
print(a)

None


In [75]:
s = 'hi, i am ...'
print(s[-1:-8:-1])

... ma 


In [86]:
[0] * 3

[0, 0, 0]